In [95]:
import pyodbc
import pandas as pd

In [96]:
def read(conn):
    print("read...")
    cursor = conn.cursor()
    cursor.execute("select * from student")
    for row in cursor:
        print(f'row = {row}')
        
    print()

In [97]:
def insert(conn,Sid,Fname,Lname):
    print("insert...")
    cursor = conn.cursor()
    cursor.execute(
        "insert into student(stdID,FirstName,LastName) values (?,?,?);",
         (Sid,Fname,Lname)   
        )
    conn.commit()
    

In [98]:
#update first_name depend on id
def update(conn,Fname,Sid):
    print("update...")
    cursor = conn.cursor()
    cursor.execute(
        "update student set FirstName = ? where stdID = ?;",
         (Fname,Sid)   
        )
    conn.commit()

In [99]:
#delete depend on id
def delete(conn,Sid):
    print("delete...")
    cursor = conn.cursor()
    cursor.execute(
        "delete from student where stdID = ?;",
         (Sid)   
        )
    conn.commit()

In [100]:
#connec to database
conn = pyodbc.connect(
        "Driver={SQL Server Native Client 11.0};"
        "Server=DESKTOP-IQ1O672\SQLEXPRESS;"
        "Database=ExaminationSystem;"
        "Trusted_Connection=yes;"
    )

In [101]:
read(conn)

read...
row = (1, 'ahmed', 'saeed')
row = (2, 'hamada', 'bahgat')



In [102]:
delete(conn,2)

delete...


In [103]:
insert(conn,2,"youssef","bahgat")

insert...


In [136]:
update(conn,"hamad",2)

update...


In [105]:
read(conn)

read...
row = (1, 'ahmed', 'saeed')
row = (2, 'hamada', 'bahgat')



In [106]:
import pdfkit

In [107]:
from fpdf import FPDF

In [160]:
def all_students_data_report():
    sql_query = pd.read_sql_query(''' 
                              select * from student
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from step 2

    df = pd.DataFrame(sql_query)
    df.to_csv (r'all_students_info.csv', index = False) # place 'r' before the path name

In [161]:
all_students_data_report()

In [146]:
def Num_students_at_each_course():
    sql_query = pd.read_sql_query(''' 
                              select c.CrsName [course Name], count(*) [number of student] from student
                              s inner join std_course sc on s.stdID =sc.stdID inner join 
                              course c on sc.crsID = c.crsID group by(c.CrsName)
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from step 2

    df = pd.DataFrame(sql_query)
    df.to_csv (r'num_students_each_course.csv', index = False) # place 'r' before the path name

In [147]:
Num_students_at_each_course()


In [168]:
def highest_grade_each_course():
    sql_query = pd.read_sql_query(''' 
                              select [student name], [course name], grade
                                from( 
                                select s.FirstName+' '+s.LastName [student name], c.CrsName [course name], se.grade,
                                    dense_rank() over (partition by c.CrsName order by se.grade desc ) as gradess
                                    from student s
                                    inner join student_exam se on s.stdID = se.stdID
                                    inner join Exam e on se.examID = e.ExamID
                                    inner join course c on e.crsID =c.crsID
                                )tmp
                                where gradess =1
                                order by gradess asc
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from step 2

    df = pd.DataFrame(sql_query)
    df.to_csv (r'highest_grade_at_each_course.csv', index = False) # place 'r' before the path name

In [169]:
highest_grade_each_course()

In [170]:
def highest_grade_each_studet_get():
    sql_query = pd.read_sql_query(''' 
                              select [student name], grade
                                from( 
                                select s.FirstName+' '+s.LastName [student name], se.grade,
                                    dense_rank() over (partition by s.stdID order by se.grade desc ) as gradess
                                    from student s
                                    inner join student_exam se on s.stdID = se.stdID
                                    inner join Exam e on se.examID = e.ExamID

                                )t
                                where gradess =1
                                order by gradess asc
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from step 2

    df = pd.DataFrame(sql_query)
    df.to_csv (r'highest_grade_each_studet_get.csv', index = False) # place 'r' before the path name

In [171]:
highest_grade_each_studet_get()